In [ ]:
#intro


In [17]:
import xarray as xr

# Load the NetCDF file
dataset = xr.open_dataset("C:\\Users\\rasag\Downloads\IMDrainfall_1901-2023 - Copy\IMDrainfall_1901-2023 - Copy.nc")

# View dataset variables
print(dataset)

# Print latitude and longitude ranges
print("Latitude Range:", dataset.LATITUDE.min().values, "to", dataset.LATITUDE.max().values)
print("Longitude Range:", dataset.LONGITUDE.min().values, "to", dataset.LONGITUDE.max().values)


<xarray.Dataset> Size: 6GB
Dimensions:    (TIME: 43829, LONGITUDE: 135, LATITUDE: 129)
Coordinates:
  * TIME       (TIME) datetime64[ns] 351kB 1901-01-01 1901-01-02 ... 2023-12-31
  * LONGITUDE  (LONGITUDE) float64 1kB 66.5 66.75 67.0 ... 99.5 99.75 100.0
  * LATITUDE   (LATITUDE) float64 1kB 6.5 6.75 7.0 7.25 ... 38.0 38.25 38.5
Data variables:
    RAINFALL   (TIME, LATITUDE, LONGITUDE) float64 6GB ...
Attributes:
    CDI:          Climate Data Interface version 1.9.9rc1 (https://mpimet.mpg...
    Conventions:  CF-1.6
    history:      Tue Jan 28 12:23:18 2025: cdo mergetime RF25_ind1901_rfp25....
    CDO:          Climate Data Operators version 1.9.9rc1 (https://mpimet.mpg...
Latitude Range: 6.5 to 38.5
Longitude Range: 66.5 to 100.0


In [58]:
# Define Andhra Pradesh's latitude and longitude bounds
import numpy as np
lat_min, lat_max = 12.0, 20.0
lon_min, lon_max = 77.0, 85.0
# time = ap_data["time"].values

# Select the Andhra Pradesh region
ap_data = dataset.sel(LATITUDE=slice(lat_max, lat_min), LONGITUDE=slice(lon_min, lon_max))

if ap_data["RAINFALL"].size == 0:
    print("Dataset is empty! Trying reversed order...")
    # ap_data = dataset.sel(LATITUDE: slice(lat_max, lat_min), LONGITUDE: slice(lon_max, lon_min))
    
    ap_data = dataset.sel(LATITUDE=slice(lat_min, lat_max), LONGITUDE=slice(lon_min, lon_max))

    print("New Filtered Shape:", ap_data["RAINFALL"].shape)

# Extract latitude, longitude, and rainfall data
lat = ap_data["LATITUDE"].values
lon = ap_data["LONGITUDE"].values
time = ap_data["TIME"].values
rainfall = ap_data["RAINFALL"].values  # Change variable name if needed
time_numeric = (time - np.datetime64(time[0])) / np.timedelta64(1, "D")
print(ap_data.values)



print(lat.shape, lon.shape)


Dataset is empty! Trying reversed order...
New Filtered Shape: (43829, 33, 33)
<bound method Mapping.values of <xarray.Dataset> Size: 382MB
Dimensions:    (TIME: 43829, LONGITUDE: 33, LATITUDE: 33)
Coordinates:
  * TIME       (TIME) datetime64[ns] 351kB 1901-01-01 1901-01-02 ... 2023-12-31
  * LONGITUDE  (LONGITUDE) float64 264B 77.0 77.25 77.5 ... 84.5 84.75 85.0
  * LATITUDE   (LATITUDE) float64 264B 12.0 12.25 12.5 12.75 ... 19.5 19.75 20.0
Data variables:
    RAINFALL   (TIME, LATITUDE, LONGITUDE) float64 382MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    CDI:          Climate Data Interface version 1.9.9rc1 (https://mpimet.mpg...
    Conventions:  CF-1.6
    history:      Tue Jan 28 12:23:18 2025: cdo mergetime RF25_ind1901_rfp25....
    CDO:          Climate Data Operators version 1.9.9rc1 (https://mpimet.mpg...>
(33,) (33,)


In [45]:
print("Available Latitudes:", dataset.LATITUDE.values)
print("Available Longitudes:", dataset.LONGITUDE.values)

Available Latitudes: [ 6.5   6.75  7.    7.25  7.5   7.75  8.    8.25  8.5   8.75  9.    9.25
  9.5   9.75 10.   10.25 10.5  10.75 11.   11.25 11.5  11.75 12.   12.25
 12.5  12.75 13.   13.25 13.5  13.75 14.   14.25 14.5  14.75 15.   15.25
 15.5  15.75 16.   16.25 16.5  16.75 17.   17.25 17.5  17.75 18.   18.25
 18.5  18.75 19.   19.25 19.5  19.75 20.   20.25 20.5  20.75 21.   21.25
 21.5  21.75 22.   22.25 22.5  22.75 23.   23.25 23.5  23.75 24.   24.25
 24.5  24.75 25.   25.25 25.5  25.75 26.   26.25 26.5  26.75 27.   27.25
 27.5  27.75 28.   28.25 28.5  28.75 29.   29.25 29.5  29.75 30.   30.25
 30.5  30.75 31.   31.25 31.5  31.75 32.   32.25 32.5  32.75 33.   33.25
 33.5  33.75 34.   34.25 34.5  34.75 35.   35.25 35.5  35.75 36.   36.25
 36.5  36.75 37.   37.25 37.5  37.75 38.   38.25 38.5 ]
Available Longitudes: [ 66.5   66.75  67.    67.25  67.5   67.75  68.    68.25  68.5   68.75
  69.    69.25  69.5   69.75  70.    70.25  70.5   70.75  71.    71.25
  71.5   71.75  72.    72.25 

In [54]:
lat.shape
# lon.shape
# rainfall.shape


(33,)

In [2]:
timesteps = 10  # Choose based on your data
X, y = [], []

In [60]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Flatten latitude, longitude, and time for LSTM
# lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')
lat_grid, lon_grid = np.meshgrid(lat, lon, indexing="ij")

lat_expanded = np.tile(lat_grid, (rainfall.shape[0], 1, 1))  # (time, 33, 33)
lon_expanded = np.tile(lon_grid, (rainfall.shape[0], 1, 1))
# data = np.column_stack((lat_grid.flatten(), lon_grid.flatten(), time_grid.flatten(), rainfall.flatten()))


time_flat = np.repeat(time_numeric, 33 * 33)  # Convert time to numerical and repeat
lat_flat = lat_expanded.flatten()
lon_flat = lon_expanded.flatten()
rainfall_flat = rainfall.flatten()

assert lat_flat.shape == lon_flat.shape == rainfall_flat.shape == time_flat.shape, "Shape mismatch after flattening!"



data = np.column_stack((time_flat, lat_flat, lon_flat, rainfall_flat))
print("Final Data Shape:", data.shape)





Final Data Shape: (47729781, 4)


In [61]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Reshape into LSTM format: (samples, timesteps, features)
# timesteps = 10  # Define number of past time steps for prediction
# X, y = [], []

In [62]:
timesteps = 10  # Define number of past time steps for prediction
X, y = [], []

In [63]:
for i in range(len(data_normalized) - timesteps):
    X.append(data_normalized[i:i+timesteps])
    y.append(data_normalized[i+timesteps][-1])  # Predict rainfall

X, y = np.array(X), np.array(y)

# Print final shapes
print("LSTM Input Shape:", X.shape)  # (samples, timesteps, features)
print("LSTM Output Shape:", y.shape)  # (samples,)

LSTM Input Shape: (47729771, 10, 4)
LSTM Output Shape: (47729771,)


In [4]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from sklearn.preprocessing import MinMaxScaler

# # Define LSTM model
# model = Sequential([
#     LSTM(50, return_sequences=True, input_shape=(timesteps, X.shape[2])),
#     LSTM(50, return_sequences=False),
#     #Dense(25, activation="relu"),
#     Dense(1)  # Predicting one value (temperature)
# ])
# #
# model.compile(optimizer="adam", loss="mse")
# model.summary()

# # Train model
# model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)


AttributeError: 'list' object has no attribute 'shape'

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

X = np.array(X, dtype=np.float32)  # Ensure numerical dtype
y = np.array(y, dtype=np.float32)

# Debugging input data
print(f"X shape before reshaping: {X.shape}")
print(f"y shape before reshaping: {y.shape}")

# Ensure X has correct shape (samples, timesteps, features)
timesteps = 10  # Number of past steps used for prediction
num_features = X.shape[1] if len(X.shape) > 1 else 1  # Ensure features exist

# Reshape if needed
if len(X.shape) == 2:
    X = X.reshape(-1, timesteps, num_features)

# Ensure y is 2D (samples, 1)
y = y.reshape(-1, 1)

print(f"X shape after reshaping: {X.shape}")  # Should be (samples, timesteps, features)
print(f"y shape after reshaping: {y.shape}")  # Should be (samples, 1)

# Check for empty dataset
if X.size == 0 or y.size == 0:
    raise ValueError("Dataset is empty after preprocessing. Check filtering steps.")

# Check for NaNs
if np.isnan(X).any() or np.isnan(y).any():
    raise ValueError("Dataset contains NaN values. Please clean your data.")

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(timesteps, X.shape[2])),
    LSTM(50, return_sequences=False),
    Dense(1)  # Predicting one value (rainfall)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

# Train model
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)


X shape before reshaping: (0,)
y shape before reshaping: (0, 1)
X shape after reshaping: (0,)
y shape after reshaping: (0, 1)


ValueError: Dataset is empty after preprocessing. Check filtering steps.